## Tarea y actividad en clase.

1. Hacer merge de la rama que trabajamos a main.
2. Crear una nueva rama que se llame `feat: tarea 5`.
3. Crear un nuevo `jupyter-notebook` llamado `challenger-experiments.ipynb` en la rama creada anteriormente
4. Hacer dos `parent experiments` con `Gradient Boost` y `Random Forest` regressors en donde cada uno tenga `child experiments` con búsqueda de hyper-parámetros. Puede usar cualquier libreraría con la que se sienta cómodo: `hyperopt`, `optuna`, `scikit-learn` (Grid Search, Random Search, Halving Search etc)
5. Registrar el modelo con la mejor métrica de los obtenidos en dichos experimentos en el `model registry` en el mismo modelo ya previamente creado `nyc-taxi-model`.
6. Asígnele el alias `challenger`
7. Descargue en la carpeta `data` el conjunto de datos correspondiente a marzo del 2024
8. Guardela en el `storage` disponible de `mlflow`
9. Use ese conjunto de datos para probarlo sobre los modelos con el alias `champion` y `challenger`
10. Obtenga la métrica de cada modelo
11. Decida si el nuevo modelo `challenger` debe ser promovido a `champion` o no. Use los criterios que usted como Data Scientis considere relevantes y justifique la respuesta.
12. Abrir un `PR` con los cambios hechos en la rama `feat: tarea 5` hacia la rama `main`.


Habrá dos entregas divididas de la siguiente manera:

1. **Trabajo en clase hoy Martes 17 de Septiembre de 2024.** Para esta entrega, hacer un commit con el siguiente mensaje `feat: entrega trabajo en clase` con los avances realizados en clase.

2. **Tarea: Viernes 20 de Septiembre de 2024 a las 19:55.** Esta entrega debe contener todo lo descrito anteriormente

In [50]:
import pandas as pd
from sklearn.feature_extraction import  DictVectorizer

In [51]:
def read_dataframe(filename):

    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

In [52]:
df_train = read_dataframe('../data/green_tripdata_2024-01.parquet')
df_val = read_dataframe('../data/green_tripdata_2024-02.parquet')

In [53]:
#For March
!curl -o ../data/green_tripdata_2024-01.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-01.parquet
!curl -o ../data/green_tripdata_2024-02.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-02.parquet
!curl -o ../data/green_tripdata_2024-03.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-03.parquet


df_train = read_dataframe('../data/green_tripdata_2024-03.parquet')
df_val = read_dataframe('../data/green_tripdata_2024-03.parquet')

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1330k  100 1330k    0     0  3382k      0 --:--:-- --:--:-- --:--:-- 3385k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1253k  100 1253k    0     0   888k      0  0:00:01  0:00:01 --:--:--  888k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1340k  100 1340k    0     0  4006k      0 --:--:-- --:--:-- --:--:-- 4012k


In [54]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [55]:
categorical = ['PU_DO']  #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']
dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [56]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [57]:
import dagshub
import mlflow


dagshub.init(url="https://dagshub.com/Pepe-Chuy/nyc-taxi-time-prediction", mlflow=True)

MLFLOW_TRACKING_URI = mlflow.get_tracking_uri()

print(MLFLOW_TRACKING_URI)

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(experiment_name="nyc-taxi-experiment")

Initialized MLflow to track repo "Pepe-Chuy/nyc-taxi-time-prediction"

Repository Pepe-Chuy/nyc-taxi-time-prediction initialized!

https://dagshub.com/Pepe-Chuy/nyc-taxi-time-prediction.mlflow


<Experiment: artifact_location='mlflow-artifacts:/f89c177353e742bf83c2fd3dab8421bb', creation_time=1726630694466, experiment_id='0', last_update_time=1726630694466, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [58]:
training_dataset = mlflow.data.from_numpy(X_train.data, targets=y_train, name="green_tripdata_2024-01")
validation_dataset = mlflow.data.from_numpy(X_val.data, targets=y_val, name="green_tripdata_2024-02")

In [59]:
from dagshub import get_repo_bucket_client
# Get a boto3.client object
s3 = get_repo_bucket_client("Pepe-Chuy/nyc-taxi-time-prediction")

# Upload file
s3.upload_file(
    Bucket="nyc-taxi-time-prediction",  # name of the repo
    Filename="../data/green_tripdata_2024-01.parquet",  # local path of file to upload
    Key="train_data.parquet",  # remote path where to upload the file
)

# Upload file
s3.upload_file(
    Bucket="nyc-taxi-time-prediction",  # name of the repo
    Filename="../data/green_tripdata_2024-02.parquet",  # local path of file to upload
    Key="eval_data.parquet",  # remote path where to upload the file
)

Client created. Use the name of the repo (nyc-taxi-time-prediction) as the name of the bucket

## Modeling

In [60]:
from sklearn.ensemble import RandomForestRegressor as rfr, GradientBoostingRegressor as gbr
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import numpy as np
from sklearn.metrics import mean_squared_error

### Random Forest

In [61]:
# Random Forest
def objective(params):
    with mlflow.start_run(nested=True):

        #model family
        mlflow.set_tag("model_family", "RandomForest")

        # Log parameters
        mlflow.log_params(params)
        
        # Train model
        rf_model = rfr(
            n_estimators=int(params['n_estimators']),
            max_depth=int(params['max_depth']),
            min_samples_split=int(params['min_samples_split']),
            min_samples_leaf=int(params['min_samples_leaf']),
            random_state=42
        )
        rf_model.fit(X_train, y_train)
        
        # Predict 
        y_pred = rf_model.predict(X_val)
        
        # RMSE
        rmse = np.sqrt(mean_squared_error(y_val, y_pred))
        
        # Log RMSE 
        mlflow.log_metric("rmse", rmse)
 
    return {'loss': rmse, 'status': STATUS_OK}

In [62]:
mlflow.sklearn.autolog()

#Params for RandomForest
search_space = {
    'n_estimators': hp.quniform('n_estimators', 10, 20, 1),
    'max_depth': hp.quniform('max_depth', 5, 10, 1),
    'min_samples_split': hp.quniform('min_samples_split', 2, 10, 1),
    'min_samples_leaf': hp.quniform('min_samples_leaf', 1, 10, 1),
}
 

In [63]:
# hyperparameter optimization
with mlflow.start_run(run_name="Father Random Forest Regressor", nested=True):
    best_params = fmin(
        fn=objective,
        space=search_space,
        algo=tpe.suggest,
        max_evals=10,
        trials=Trials()
    )
    
    # Log best model
    mlflow.log_params(best_params)

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

2024/09/20 17:54:33 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'

2024/09/20 17:54:37 WARNING mlflow.utils.requirements_utils: Found lz4 version (4.0.2+dfsg) contains a local version label (+dfsg). MLflow logged a pip requirement for this package as 'lz4==4.0.2' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.

2024/09/20 17:54:40 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID b0a95f8f0af44990a4c16aec8077106f. Failed operations: [RestException("INVALID_PARAMETER_VALUE: Response: {\'error_code\': \'INVALID_PARAMETER_VALUE\'}")]')]

2024/09/20 17:54:40 INFO mlflo

 10%|█         | 1/10 [00:08<01:13,  8.21s/trial, best loss: 5.345092572070841]

2024/09/20 17:54:41 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'

2024/09/20 17:54:45 WARNING mlflow.utils.requirements_utils: Found lz4 version (4.0.2+dfsg) contains a local version label (+dfsg). MLflow logged a pip requirement for this package as 'lz4==4.0.2' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.

2024/09/20 17:54:48 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID 3427bbd42fed4b00a9ed8e89c99ca4a3. Failed operations: [RestException("INVALID_PARAMETER_VALUE: Response: {\'error_code\': \'INVALID_PARAMETER_VALUE\'}")]')]

2024/09/20 17:54:48 INFO mlflo

 20%|██        | 2/10 [00:16<01:04,  8.07s/trial, best loss: 5.322386203535303]

...

2024/09/20 17:55:34 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'

2024/09/20 17:55:37 WARNING mlflow.utils.requirements_utils: Found lz4 version (4.0.2+dfsg) contains a local version label (+dfsg). MLflow logged a pip requirement for this package as 'lz4==4.0.2' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.

2024/09/20 17:55:40 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID 000d98b9050f401ea18073a13de398ef. Failed operations: [RestException("INVALID_PARAMETER_VALUE: Response: {\'error_code\': \'INVALID_PARAMETER_VALUE\'}")]')]

2024/09/20 17:55:40 INFO mlflo

 90%|█████████ | 9/10 [01:07<00:07,  7.46s/trial, best loss: 5.322386203535303]

2024/09/20 17:55:41 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'

2024/09/20 17:55:44 WARNING mlflow.utils.requirements_utils: Found lz4 version (4.0.2+dfsg) contains a local version label (+dfsg). MLflow logged a pip requirement for this package as 'lz4==4.0.2' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.

2024/09/20 17:55:47 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID 3178a69c93ed4cecb6045712b16c5639. Failed operations: [RestException("INVALID_PARAMETER_VALUE: Response: {\'error_code\': \'INVALID_PARAMETER_VALUE\'}")]')]

2024/09/20 17:55:47 INFO mlflo

100%|██████████| 10/10 [01:14<00:00,  7.50s/trial, best loss: 5.322386203535303]


2024/09/20 17:55:48 INFO mlflow.tracking._tracking_service.client: 🏃 View run Father Random Forest Regressor at: https://dagshub.com/Pepe-Chuy/nyc-taxi-time-prediction.mlflow/#/experiments/0/runs/fb744d48590d4b239f5c77e4196642ac.
2024/09/20 17:55:48 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/Pepe-Chuy/nyc-taxi-time-prediction.mlflow/#/experiments/0.


In [64]:
best_params

{'max_depth': 10.0,
 'min_samples_leaf': 5.0,
 'min_samples_split': 4.0,
 'n_estimators': 15.0}

In [66]:
run_id = "3427bbd42fed4b00a9ed8e89c99ca4a3"
run_uri = f"runs:/{run_id}/model"
 
result = mlflow.register_model(
    model_uri=run_uri,
    name="nyc-taxi-model-rf-march"
)

Successfully registered model 'nyc-taxi-model-rf-march'.
2024/09/20 17:59:27 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: nyc-taxi-model-rf-march, version 1
Created version '1' of model 'nyc-taxi-model-rf-march'.


### GradientBoosting

In [67]:
def objective_gb(params):
    with mlflow.start_run(nested=True):
        # Model family
        mlflow.set_tag("model_family", "GradientBoosting")
        
        # Log GB parameters
        mlflow.log_params(params)
        
        # Train model
        gb_model = gbr(
            n_estimators=int(params['n_estimators']),
            max_depth=int(params['max_depth']),
            min_samples_split=int(params['min_samples_split']),
            min_samples_leaf=int(params['min_samples_leaf']),
            learning_rate=float(params['learning_rate']),
            random_state=42
        )
        gb_model.fit(X_train, y_train)
        
        # Predict
        y_pred = gb_model.predict(X_val)
        
        # RMSE
        rmse = np.sqrt(mean_squared_error(y_val, y_pred))
        
        # Log RMSE
        mlflow.log_metric("rmse", rmse)
 
    return {'loss': rmse, 'status': STATUS_OK}
 

In [68]:

mlflow.sklearn.autolog()

# PArams for gb
search_space = {
    'n_estimators': hp.quniform('n_estimators', 10, 30, 1),
    'max_depth': hp.quniform('max_depth', 1, 3, 1),
    'min_samples_split': hp.quniform('min_samples_split', 2, 5, 1),
    'min_samples_leaf': hp.quniform('min_samples_leaf', 1, 4, 1),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.1)
}
 

In [69]:
# hyperparameter optimization
with mlflow.start_run(run_name="Father Gradient Boosting", nested=True):
    best_params = fmin(
        fn=objective_gb,
        space=search_space,
        algo=tpe.suggest,
        max_evals=10,
        trials=Trials()
    )
    
    # Log best model
    mlflow.log_params(best_params)

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

2024/09/20 17:59:34 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'

2024/09/20 17:59:37 WARNING mlflow.utils.requirements_utils: Found lz4 version (4.0.2+dfsg) contains a local version label (+dfsg). MLflow logged a pip requirement for this package as 'lz4==4.0.2' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.

2024/09/20 17:59:39 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID d5a37dfd0d754b868a421e5182ce1064. Failed operations: [RestException("INVALID_PARAMETER_VALUE: Response: {\'error_code\': \'INVALID_PARAMETER_VALUE\'}")]')]

2024/09/20 17:59:40 INFO mlflo

 10%|█         | 1/10 [00:06<01:02,  6.92s/trial, best loss: 6.631532375674564]

2024/09/20 17:59:41 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'

2024/09/20 17:59:44 WARNING mlflow.utils.requirements_utils: Found lz4 version (4.0.2+dfsg) contains a local version label (+dfsg). MLflow logged a pip requirement for this package as 'lz4==4.0.2' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.

2024/09/20 17:59:47 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID ea5cf49503b1401288905133949a81ad. Failed operations: [RestException("INVALID_PARAMETER_VALUE: Response: {\'error_code\': \'INVALID_PARAMETER_VALUE\'}")]')]

2024/09/20 17:59:47 INFO mlflo

 20%|██        | 2/10 [00:14<00:57,  7.19s/trial, best loss: 6.631532375674564]

...

2024/09/20 18:00:32 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'

2024/09/20 18:00:35 WARNING mlflow.utils.requirements_utils: Found lz4 version (4.0.2+dfsg) contains a local version label (+dfsg). MLflow logged a pip requirement for this package as 'lz4==4.0.2' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.

2024/09/20 18:00:37 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID f4f1515ee0de4daaa469d5f1699900c4. Failed operations: [RestException("INVALID_PARAMETER_VALUE: Response: {\'error_code\': \'INVALID_PARAMETER_VALUE\'}")]')]

2024/09/20 18:00:38 INFO mlflo

 90%|█████████ | 9/10 [01:04<00:07,  7.27s/trial, best loss: 5.8390128802759556]

2024/09/20 18:00:39 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'

2024/09/20 18:00:42 WARNING mlflow.utils.requirements_utils: Found lz4 version (4.0.2+dfsg) contains a local version label (+dfsg). MLflow logged a pip requirement for this package as 'lz4==4.0.2' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.

2024/09/20 18:00:45 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID 41a9da234b624bec80de2080e80b0e89. Failed operations: [RestException("INVALID_PARAMETER_VALUE: Response: {\'error_code\': \'INVALID_PARAMETER_VALUE\'}")]')]

2024/09/20 18:00:45 INFO mlflo

100%|██████████| 10/10 [01:12<00:00,  7.23s/trial, best loss: 5.8390128802759556]


2024/09/20 18:00:46 INFO mlflow.tracking._tracking_service.client: 🏃 View run Father Gradient Boosting at: https://dagshub.com/Pepe-Chuy/nyc-taxi-time-prediction.mlflow/#/experiments/0/runs/bb9183ba2dfc4ef0908276491158e4ae.
2024/09/20 18:00:46 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/Pepe-Chuy/nyc-taxi-time-prediction.mlflow/#/experiments/0.


In [76]:
run_id = "000d98b9050f401ea18073a13de398ef"
run_uri = f"runs:/{run_id}/model"
 
result = mlflow.register_model(
    model_uri=run_uri,
    name="nyc-taxi-mode-won-timermse"
)

Successfully registered model 'nyc-taxi-mode-won-timermse'.
2024/09/20 18:13:24 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: nyc-taxi-mode-won-timermse, version 1
Created version '1' of model 'nyc-taxi-mode-won-timermse'.


# Winner

Finally, the champion was "000d98b9050f401ea18073a13de398ef," better known as "nyc-taxi-mode-won-timermse." Although its RMSE may not be the global best, it receives a downgrade based on the best one according to the time it took to run. Nevertheless, this model had the best overall performance.

If time didn’t matter, the champion would be "nyc-taxi-mode-gbl" V4.